**Nome: Felipe Bizzo ----------------------------------- RA: 1553518**

In [ ]:
!pip install pyromat

In [2]:
import pyromat as pm
import numpy as np

pm.config["unit_pressure"] = "kPa"
pm.config["def_p"] = 100

mp_water = pm.get("mp.H2O") # <-- for multi-phase water properties

In [5]:
p1 = 10 # <-- saida do condensador
p2 = 800 # <-- saida da bomba
p3 = 200 # <-- saida do gerador
p4 = 3500 # <-- entrada da turbina/saida da caldeira
p5 = p4 # <-- saida da turbina
T5 = 400 + 273.15 # K <-- saida da turbina
p6 = p1 # <-- entrada no condensador/saida da turbina

Primeiro vamos considerar a bomba de baixa pressão

In [19]:
h1 = mp_water.hs(p=p1)[0]

v1 = 1/mp_water.ds(p=p1)[0]

w_pump1 = v1*(p2-p1)

print(f"Trabalho gerado na bomba 1: {round(float(w_pump1),1)} kJ/kg")

h2 = h1+w_pump1

Trabalho gerado na bomba 1: 0.8 kJ/kg


Agora vamos considerar a turbina:

In [18]:
h5 = mp_water.h(p=p5, T=T5)
s5 = mp_water.s(p=p5, T=T5)

s6 = s5

T6, x6 = mp_water.T_s(s=s6, p=p6, quality=True)
h6 = mp_water.h(x=x6, p=p6)

print(f"Qualidade do vapor de água: {round(float(x6),4)}")

Qualidade do vapor de água: 0.8258


Agora, considerando o aquecedor de água:

In [9]:
h3 = mp_water.hs(p=p3)[0]

y = (h2-h3)/(h2-h6)
print(f"y = {round(float(y),4)}")

Podemos calcular agora o trabalho gerado na turbina:

In [17]:
p7 = p1
s7 = s5
T7, x7 = mp_water.T_s(s=s7, p=p7, quality=True)
h7 = mp_water.h(x=x7, p=p7)
w_t = h5 - y*h6 - (1-y)*h7
print(f"Trabalho gerado na turbina: {round(float(w_t),1)} kJ/kg")

Trabalho gerado na turbina: 1055.9 kJ/kg


Considerando a bomba de alta pressão:

In [16]:
v3 = 1/mp_water.ds(p=p3)[0]
w_pump2 = v3*(p4-p3)
print(f"Trabalho requerido pela bomba 2: {round(float(w_pump2),1)} kJ/kg")

Trabalho requerido pela bomba 2: 3.5 kJ/kg


Finalmente, considerando a caldeira:

In [15]:
h4 = h3 + w_pump2
q_H = h5-h4
print(f"Calor gerado pela caldeira: {round(float(q_H),1)} kJ/kg")

Calor gerado pela caldeira: 2715.0 kJ/kg


Podemos agora calcular a eficiência térmica do sistema com: $$\eta_{th}=\frac{w_{net}}{q_H}$$

In [20]:
eta_th = (w_t - w_pump1*(1-y) - w_pump2)/q_H*100
print(f"A eficiência térmica é: {round(float(eta_th),2)}%")

A eficiência térmica é: 38.74%
